# SDFusion: Text-guided Generation (txt2shape)

### TODO: add sample results or teaser images

In [1]:
!git clone https://github.com/ScheznikL/SDFusion.git
%cd SDFusion
!ls

Cloning into 'SDFusion'...
remote: Enumerating objects: 1976, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 1976 (delta 52), reused 81 (delta 34), pack-reused 1859 (from 1)
Receiving objects: 100% (1976/1976), 35.93 MiB | 13.69 MiB/s, done.
Resolving deltas: 100% (813/813), done.
/content/SDFusion
configs		    demo_img2shape.ipynb	  external   options	   train.py
dataset_info_files  demo_mm2shape.ipynb		  launchers  preprocess    utils
datasets	    demo_txt2shape.ipynb	  LICENSE    README.md
demo_data	    demo_uncond_shape_comp.ipynb  models     setup_env.sh


In [2]:
# first set up which gpu to use
import os
gpu_ids = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_ids}"

In [3]:
#exit()
#How to run a Python script in a '.py' file from a Google Colab notebook?
from models.base_model import create_model

In [4]:
!pip install h5py joblib trimesh scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 15.7 MB/s eta 0:00:00


In [5]:
!pip install --upgrade PyMCubes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 7.2 MB/s eta 0:00:00


In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("2.1.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-2kavpzc4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-2kavpzc4
  Running command git checkout -q 75ebeeaea0908c5527e7b1e305fbc7681382db47
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 75ebeeaea0908c5527e7b1e305fbc7681382db47
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# import libraries
import numpy as np
from IPython.display import Image as ipy_image
from IPython.display import display
from termcolor import colored, cprint

import torch
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
import torchvision.utils as vutils

from models.base_model import create_model
from utils.util_3d import render_sdf, render_mesh, sdf_to_mesh, save_mesh_as_gif

%load_ext autoreload
%autoreload 2

# New section

In [ ]:
!pip install omegaconf

In [9]:
# options for the model. please check `utils/demo_util.py` for more details
from utils.demo_util import SDFusionText2ShapeOpt

seed = 2023
opt = SDFusionText2ShapeOpt(gpu_ids=gpu_ids, seed=seed)
device = opt.device

[*] SDFusionText2ShapeOption initialized.


In [10]:
!mkdir saved_ckpt

In [11]:
# SDFusion: text-guided shape generation (txt2shape)
!wget https://uofi.box.com/shared/static/vyqs6aex3rwbgxweyl3qh21c8p6vu33f.pth -O saved_ckpt/sdfusion-txt2shape.pth

--2025-01-05 23:23:38--  https://uofi.box.com/shared/static/vyqs6aex3rwbgxweyl3qh21c8p6vu33f.pth
Resolving uofi.box.com (uofi.box.com)... 74.112.186.157
Connecting to uofi.box.com (uofi.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/vyqs6aex3rwbgxweyl3qh21c8p6vu33f.pth [following]
--2025-01-05 23:23:38--  https://uofi.box.com/public/static/vyqs6aex3rwbgxweyl3qh21c8p6vu33f.pth
Reusing existing connection to uofi.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uofi.app.box.com/public/static/vyqs6aex3rwbgxweyl3qh21c8p6vu33f.pth [following]
--2025-01-05 23:23:39--  https://uofi.app.box.com/public/static/vyqs6aex3rwbgxweyl3qh21c8p6vu33f.pth
Resolving uofi.app.box.com (uofi.app.box.com)... 74.112.186.157
Connecting to uofi.app.box.com (uofi.app.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [12]:
# VQVAE's checkpoint
!wget https://uofi.box.com/shared/static/zdb9pm9wmxaupzclc7m8gzluj20ja0b6.pth -O saved_ckpt/vqvae-snet-all.pth

--2025-01-05 23:24:22--  https://uofi.box.com/shared/static/zdb9pm9wmxaupzclc7m8gzluj20ja0b6.pth
Resolving uofi.box.com (uofi.box.com)... 74.112.186.157
Connecting to uofi.box.com (uofi.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/zdb9pm9wmxaupzclc7m8gzluj20ja0b6.pth [following]
--2025-01-05 23:24:23--  https://uofi.box.com/public/static/zdb9pm9wmxaupzclc7m8gzluj20ja0b6.pth
Reusing existing connection to uofi.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://uofi.app.box.com/public/static/zdb9pm9wmxaupzclc7m8gzluj20ja0b6.pth [following]
--2025-01-05 23:24:23--  https://uofi.app.box.com/public/static/zdb9pm9wmxaupzclc7m8gzluj20ja0b6.pth
Resolving uofi.app.box.com (uofi.app.box.com)... 74.112.186.157
Connecting to uofi.app.box.com (uofi.app.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcl

In [13]:
# initialize SDFusion model
ckpt_path = 'saved_ckpt/sdfusion-txt2shape.pth'
opt.init_model_args(ckpt_path=ckpt_path)

SDFusion = create_model(opt)
cprint(f'[*] "{SDFusion.name()}" loaded.', 'cyan')

Working with z of shape (1, 3, 16, 16, 16) = 12288 dimensions.


/content/SDFusion/models/model_utils.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(vq_ckpt, map_location=map_fn)


[*] VQVAE: weight successfully load from: saved_ckpt/vqvae-snet-all.pth


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/content/SDFusion/models/sdfusion_txt2shape_model.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(ckpt, map_location=map_fn)


[*] weight successfully load from: saved_ckpt/sdfusion-txt2shape.pth
[*] setting ddim_steps=100
[*] Model has been created: SDFusion-Text2Shape-Model
[*] "SDFusion-Text2Shape-Model" loaded.


## SDFusion: text-guided generation (txt2shape)

In [19]:

# txt2shape
out_dir = 'demo_results'
if not os.path.exists(out_dir): os.makedirs(out_dir)

# change the input text here to generate different chairs/tables!
input_txt = "A round red color chair with four legs 0.5 meters in length each"

ngen = 1 # number of generated shapes
ddim_steps = 100
ddim_eta = 0.
uc_scale = 3.

sdf_gen = SDFusion.txt2shape(input_txt=input_txt, ngen=ngen, ddim_steps=ddim_steps, ddim_eta=ddim_eta, uc_scale=uc_scale)

mesh_gen = sdf_to_mesh(sdf_gen)

'''# Save meshes to .obj and/or .ply
for i, mesh in enumerate(mesh_gen):
    verts, faces = mesh.verts_packed(), mesh.faces_packed()
    # Save to .obj format
    obj_path = f"{out_dir}/shape_{i}_{input_txt.replace(' ', '_')}.obj"
    save_obj(obj_path, verts, faces)
    print(f"Saved OBJ to {obj_path}")'''

# vis as gif
#gen_name = f'{out_dir}/txt2shape-{input_txt}.gif'
#save_mesh_as_gif(SDFusion.renderer, mesh_gen, nrow=3, out_name=gen_name)

#print(f'Input: "{input_txt}"')
#for name in [gen_name]:
#    display(ipy_image(name))

Data shape for DDIM sampling is (1, 3, 16, 16, 16), eta 0.0
Running DDIM Sampling with 100 timesteps


DDIM Sampler: 100%|██████████| 100/100 [00:37<00:00,  2.69it/s]


'# Save meshes to .obj and/or .ply\nfor i, mesh in enumerate(mesh_gen):\n    verts, faces = mesh.verts_packed(), mesh.faces_packed()\n    # Save to .obj format\n    obj_path = f"{out_dir}/shape_{i}_{input_txt.replace(\' \', \'_\')}.obj"\n    save_obj(obj_path, verts, faces)\n    print(f"Saved OBJ to {obj_path}")'

In [28]:
#from pytorch3d.io import save_mesh
from pathlib import Path

mesh_path = Path(out_dir) / f"NEWshape_{i}_{input_txt.replace(' ', '_')}.ply"

from pytorch3d.io import IO
IO().save_mesh(mesh_gen, mesh_path,binary=False,colors_as_uint8=True)

# Directory for saving meshes
#out_dir = 'demo_results'
#Path(out_dir).mkdir(parents=True, exist_ok=True)

# Iterate through generated meshes and save each one
'''for i, mesh in enumerate(mesh_gen):
    # Ensure the mesh has only one element
    #mesh = mesh.extend(1)  # Create a single-element Meshes object

    # Define the output path


    # Save the mesh with textures included
    save_mesh(mesh, mesh_path, binary=True, include_textures=True)
    print(f"Saved mesh to {mesh_path}")'''

'for i, mesh in enumerate(mesh_gen):\n    # Ensure the mesh has only one element\n    #mesh = mesh.extend(1)  # Create a single-element Meshes object\n\n    # Define the output path\n    \n\n    # Save the mesh with textures included\n    save_mesh(mesh, mesh_path, binary=True, include_textures=True)\n    print(f"Saved mesh to {mesh_path}")'

In [29]:
import os
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    DirectionalLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)



In [30]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

--2025-01-06 00:25:21--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘plot_image_grid.py’

plot_image_grid.py  100%[===================>]   1.57K  --.-KB/s    in 0s      

2025-01-06 00:25:21 (29.3 MB/s) - ‘plot_image_grid.py’ saved [1608/1608]



In [ ]:
from pytorch3d.io import save_ply

verts = mesh_gen.verts_packed()
faces = mesh_gen.faces_packed()
verts_rgb = mesh_gen.textures.verts_features_packed()

# Save to PLY
ply_path = Path(out_dir) / "textured_mesh.ply"
save_ply(ply_path, verts, faces, verts_rgb=verts_rgb)
print(f"Saved PLY with vertex colors to {ply_path}")

## TODO

In [32]:
#images = SDFusion.renderer(mesh)

img_comb = render_mesh(SDFusion.renderer, mesh_gen, norm=False)
plt.figure(figsize=(10, 10))
plt.imshow(img_comb[0, ..., :3].cpu().numpy())
plt.axis("off");
'''
plt.figure(figsize=(7,7))
texture_image=mesh_gen.textures.verts_features_packed()
plt.imshow(texture_image.squeeze().cpu().numpy())
plt.axis("off");'''

AttributeError: 'TexturesVertex' object has no attribute 'squeeze'

<Figure size 700x700 with 0 Axes>

In [21]:
from pytorch3d.io import save_obj, save_ply

 # Assuming each mesh has a `textures` attribute with a Textures object
for i, mesh in enumerate(mesh_gen):
    verts, faces = mesh.verts_packed(), mesh.faces_packed()

    # Save geometry and textures to .obj format
    if hasattr(mesh, 'textures') and mesh.textures is not None:
        vertex_colors = mesh.textures.verts_features_packed()  # Extract vertex colors

        # Save to .obj without UVs but with vertex colors
        obj_path = f"{out_dir}/shape_{i}_{input_txt.replace(' ', '_')}_textured.obj"
        save_obj(obj_path, verts, faces)  # OBJ doesn't natively support vertex colors
        print(f"Saved OBJ to {obj_path} (vertex colors not directly supported in OBJ)")

        # Save to .ply with vertex colors
        ply_path = f"{out_dir}/shape_{i}_{input_txt.replace(' ', '_')}_textured.ply"
        save_ply(ply_path, verts, faces, verts_rgb=vertex_colors)
        print(f"Saved PLY to {ply_path} (with vertex colors)")

    else:
        # Save without textures
        obj_path = f"{out_dir}/shape_{i}_{input_txt.replace(' ', '_')}.obj"
        save_obj(obj_path, verts, faces)
        print(f"Saved OBJ (no textures) to {obj_path}")


Saved OBJ to demo_results/shape_0_A_round_red_color_chair_with_four_legs_0.5_meters_in_length_each_textured.obj (vertex colors not directly supported in OBJ)


TypeError: save_ply() got an unexpected keyword argument 'verts_rgb'